# Is it a wood screw?
This is the first lecture notebook from fastai's Practical Deep Learning for Coders 2022.


https://course.fast.ai/Lessons/lesson1.html

In [1]:
%pip install -Uqq fastai duckduckgo_search

Note: you may need to restart the kernel to use updated packages.


## Step 0: Learn about the subject

* Types of wood screws: https://www.thehandymansdaughter.com/wood-screws/
* Metal vs Wood screws: https://www.finepowertools.com/woodworking/metal-screw-vs-wood-screw/

## Step 1: Gather the data

In [5]:
from pkgutil import ImpImporter
from duckduckgo_search import ddg_images
from fastcore.all import *

def search_images(term, max_images=10):
    print(f"Searching for {term}")
    return L(ddg_images(term, max_results=max_images)).itemgot('image')

In [3]:
%time
urls = search_images('sheet metal screws', 20)
urls[0]

CPU times: user 1 µs, sys: 1e+03 ns, total: 2 µs
Wall time: 3.34 µs
Searching for sheet metal screws


'https://d3d71ba2asa5oz.cloudfront.net/12028546/images/prsms188ss1.jpg'

In [7]:
from fastai.vision.all import *
searches = 'wood','sheet-metal'
path = Path('screws')
for o in searches:
    dest = (path/o)
    dest.mkdir(exist_ok=True,parents=True)
    download_images(dest, urls=search_images(f"{o} screws"))
    # resize images in-situ
    resize_images(dest, max_size=400, dest=dest)

Searching for wood screws
Searching for sheet-metal screws


In [10]:
failed = verify_images(get_image_files(path))
failed.map(Path.unlink)
len(failed)

0

## Step 2: Train your model

Create dataloders `dls` from `path` with batch size of 32 images

In [11]:
dls = DataBlock(
    blocks=(ImageBlock,CategoryBlock),
    get_items=get_image_files,
    splitter=RandomSplitter(0.3, seed=50),
    get_y=parent_label,
    item_tfms=[Resize(192, method='squish')]
).dataloaders(path, bs=32)

* `blocks=(ImageBlock,CategoryBlock)` means the inputs are Images and the output are categories ("wood", or "sheet-metal")
* `get_items=get_image_files` means use `get_image_files` function to find the input to our models


## Step 3: Use your model